In [1]:
import pandas as pd
import numpy as np
import requests
from concurrent.futures import ThreadPoolExecutor
import sys
#from tqdm.notebook import tqdm
sys.path.append('/My Drive/Excels útiles/')
from functions import *

In [3]:
sellers_id = list(pd.read_csv('G:\\My Drive\Proyectos\\[CL Scrape] Full Meli Scrape\\20220901\\all_sellers_id.csv')['meli_seller_id'])
sellers_id;

In [6]:
requests.get(f'https://api.mercadolibre.com/sites/MLC/search?seller_id=494294258').json()['paging']['total']

1941

In [4]:
def scrape_3(list_seller_id:list) -> pd.DataFrame:
    results = pd.DataFrame({}, columns=['meli_seller_id', 'meli_seller_name', 'historical_sales', 'sales_last60days', 'total_skus'])
    for i in list_seller_id:
        #print(i)
        req = reintentos_request(f'https://api.mercadolibre.com/sites/MLC/search?seller_id={i}').json()
        
        try:
            meli_seller_id = req['seller']['id']
        except:
            meli_seller_id = np.nan
        
        try:
            meli_seller_name = req['seller']['nickname']
        except:
            meli_seller_name= np.nan

        try:
            historical_sales = req['seller']['seller_reputation']['transactions']['total']
        except:
            historical_sales= np.nan

        try:
            sales_last60days = req['seller']['seller_reputation']['metrics']['sales']['completed']
        except:
            sales_last60days= np.nan
        
        try:
            total_skus = req['paging']['total']
        except:
            total_skus = np.nan
            
        new_row = pd.DataFrame({
            'meli_seller_id'  : meli_seller_id,
            'meli_seller_name': meli_seller_name,
            'historical_sales': historical_sales,
            'sales_last60days': sales_last60days,
            'total_skus'      : total_skus
        }, columns=results.columns, index=[0])
        results = pd.concat([results, new_row],  ignore_index=True)
    return results
#test
scrape_3(sellers_id[:10])

,meli_seller_id,meli_seller_name,historical_sales,sales_last60days,total_skus
0,18648,FRAMIRE,1,1,1
1,26407,ULTRA_SOLUCIONES,11,7,29
2,27010,PALOSKI2006,3,3,4
3,64602,NACHO_320,2,0,1
4,92871,PERFECT PULCHER,2896,1141,92
5,98756,TICOM STORE,25,22,46
6,199283,HONIGCHILE,17,15,12
7,204167,MERCADO1453,18,18,17
8,205236,NOTEBOOK CL,34,33,7
9,205569,AUCTIONER,397,90,1536


In [5]:
empty_sheet = pd.DataFrame({}, columns=['meli_seller_id', 'meli_seller_name', 'historical_sales', 'sales_last60days', 'total_skus'])

values_list = [sellers_id[element:element+100] for element in range(0,149600, 100)]
values_list

with ThreadPoolExecutor(max_workers=len(values_list)*2) as executor:
    results = list(executor.map(scrape_3, values_list))

falló un request(1). https://api.mercadolibre.com/sites/MLC/search?seller_id=1080413360


In [6]:
for i in results:
    empty_sheet = pd.concat([empty_sheet, i], ignore_index=True)
empty_sheet

,meli_seller_id,meli_seller_name,historical_sales,sales_last60days,total_skus
0,18648,FRAMIRE,1,1,1
1,26407,ULTRA_SOLUCIONES,11,7,29
2,27010,PALOSKI2006,3,3,4
3,64602,NACHO_320,2,0,1
4,92871,PERFECT PULCHER,2896,1141,92
...,...,...,...,...,...
45221,1186351472,JBINTERNATIONALIMPORTSPAJB,0,0,2
45222,1186522800,MODAJEANNA20220825212328,0,0,7
45223,1186897781,GADGETSSTOREPUNTOCL,0,0,11
45224,1188942355,ALCYON SPA,1,1,23


In [7]:
empty_sheet.sort_values(by='sales_last60days', ascending=False).to_excel("all_meli_sellers_data.xlsx")